In [1]:
import os
import glob
import json
import Utils.Chat_GPT_Funcs as GPT
import concurrent.futures
import os
import string

# Load in the 343 Personas that mimic the study done

In [25]:
# Open the JSON file in read mode and load its contents
file_path = "new_personas_343.json"
with open(file_path, "r") as json_file:
    personas = json.load(json_file)
    
personas = personas[:3]
print("Total Personas:", len(personas))

Total Personas: 3


In [26]:
#personas[75]

# Load the survey questions
    - Using questions from phone interview

In [27]:
class Question:
    def __init__(self, number, question, possible_responses):
        self.number = number
        self.question = question
        self.possible_responses = possible_responses

    def format_for_gpt(self):
        formatted_question = self.question + '\n'
        formatted_responses = '\n'.join(self.possible_responses)

        return formatted_question + formatted_responses

In [28]:
questions = [
    Question(1, "Do you plan to vote in the 2022 midterm election?", ["Yes", "No", "Neither"]),
    Question(2, "At this point in time, do you approve or disapprove of President Joe Biden’s performance in office?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(3, "Do you approve or disapprove of Biden’s student loan forgiveness plan?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(4, "At this point in time, do you approve or disapprove of the performance of the US Congress?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(5, "When electing the United States president, do you think we should amend the Constitution to allow the candidate who receives the most votes to win, or do you think we should keep the current system where the candidate who wins the Electoral College vote wins?", ["Amend the Constitution", "Keep the current system", "Neither Amend/Keep"]),
    Question(6, "At this point in time, do you approve or disapprove of Governor Greg Gianforte’s job performance?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(7, "Gianforte’s administration recently attempted to implement a rule banning transgender people from changing their designated sex status on their birth certificate. Do you think transgender people should be able to change the sex on their birth certificate?", ["Yes", "No", "Neither Yes/No"]),
    Question(8, "Are you in US House District 1 or 2?", ["District 1", "District 2", "Don't Know"]),
    Question(9, "If you had to choose today, which of the following candidates would you choose to support for House Representative in District 1?", ["Zinke (R)","Tranel (D)", "Lamb (L)", "Don't Know"]),
    Question(10, "At this point in time, do you approve or disapprove of Congressperson Matt Rosendale’s job performance?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(11, "If you had to choose today, which of the following candidates would you choose to support for House Representative in District 2?", ["Rosendale (R)","Ronning (D)", "Rankin (L)", "Buchanan (I)", "Don't Know"]),
    Question(12, "At this point in time, do you approve or disapprove of Senator Steve Daines’ job performance?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(13, "At this point in time, do you approve or disapprove of Senator Jon Tester’s job performance?", ["Approve", "Disapprove", "Neither Approve/Disapprove"]),
    Question(14, "Do you trust your representatives’ ability to govern efficiently and in a non-divisive manner?", ["Trust", "Distrust", "Neither Trust/Distrust"]),
    Question(15, "Do you trust your local police or sheriff’s department?", ["Trust", "Distrust", "Neither Trust/Distrust"]),
    Question(16, "Have you heard of the Indian Education for All law?", ["Yes", "No", "Neither Yes/No"]),
    Question(17, "The Indian Education for All law is based on a Montana constitutional requirement to encourage learning about American Indian culture and history. Do you believe the money from this law is being used effectively to educate the general student body within Montana’s school system?", ["Yes", "No", "Neither Yes/No"]),
    Question(18, "Do you agree or disagree that there is a mental health crisis in Montana?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(19, "Do you agree or disagree that more fun indoor activities could help the depression rate in Montana?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(20, "Do you agree or disagree that businesses in your community are struggling to find employees?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(21, "Do you agree or disagree that current Montana agricultural practices are environmentally sustainable?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(22, "Do you agree or disagree that Montana officials should work to protect public lands?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(23, "Do you agree or disagree with the Supreme Court's decision to overturn Roe v. Wade?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(24, "When it comes to abortion issues, would you consider yourself to be pro-life or pro-choice?", ["Pro-life", "Pro-choice", "Neither Pro-life/Pro-choice"]),
    Question(25, "Do you agree or disagree with the efforts to remove potentially controversial books from public schools?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(26, "Do you agree or disagree that teachers with a clean background check and training should be allowed to carry a gun in the classroom?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(27, "Do you agree or disagree with using the death penalty as punishment for convicted murder?", ["Agree", "Disagree", "Neither Agree/Disagree"]),
    Question(28, "Do you trust mainstream media?", ["Trust", "Distrust", "Neither Trust/Distrust"])
]


# Initialize Survey Responses
    - A survey response will each have
        - Persona responding
        - Dictionary of responses mapped to the question number
            - Doing dictionary instead of list just in case of a failure of gpt to answer a question
            - The indices will have correct values

In [29]:
import threading
class SurveyResponse:
    def __init__(self, person):
        self.person = person
        self.survey_responses = {}
        self.lock = threading.Lock()  # Create a lock for synchronization


    def ask_question(self, question):
        global total_prompt_tokens, total_completion_tokens
        # Load the Persona role and question
        print("got here, qn", question.number)
        persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(self.person))
        question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question.format_for_gpt())
        
        #print(persona_role, question_in)
        try:
            resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.25, tokens = 300)
            print("GPT ran")
        except:
            resp = ""
            usage = 0
    
        total_completion_tokens += usage['completion_tokens']
        total_prompt_tokens += usage['prompt_tokens']
    
        with self.lock:  # Acquire the lock before modifying the dictionary
            print(resp)
            self.survey_responses[question.number] = resp
        print("sv",self.survey_responses[question.number])
        
    def __str__(self):
        response_str = f"Survey Response for: {self.person}\n"
        response_str += "Survey Responses:\n"
        for question_number, response in self.survey_responses.items():
            response_str += f"Question {question_number}: {response}\n"
        return response_str

In [30]:
SURVEY_RESPONSES = []
for p in personas:
    SURVEY_RESPONSES.append(SurveyResponse(p))
print("Total Surveys:", len(SURVEY_RESPONSES))

Total Surveys: 3


# Conduct Survey to Personas

In [31]:
qc = 0

total_prompt_tokens = 0
total_completion_tokens = 0

# def ask_question_threaded(SURVEY_RESPONSE, question):
#     SURVEY_RESPONSE.ask_question(question)
    
def ask_question_threaded(SURVEY_RESPONSE, question):  
    global total_prompt_tokens, total_completion_tokens
    # Load the Persona role and question
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(SURVEY_RESPONSE.person))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question.format_for_gpt())

    #print(persona_role, question_in)
    try:
        resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.25, tokens = 300)
    except:
        resp = ""
        usage = 0

    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']

    SURVEY_RESPONSE.survey_responses[str(question.number)] = resp

for question in questions:
#     if qc == 1:
#         break
#     qc += 1
    # Create a thread pool executor with 10 workers
    print(f"Starting Question: {str(question.number)}\n")
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        # Submit tasks to the executor for each persona
        futures = [executor.submit(ask_question_threaded, SURVEY_RESPONSE, question) for SURVEY_RESPONSE in SURVEY_RESPONSES]

        # Wait for all tasks to complete
        concurrent.futures.wait(futures)

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

Starting Question: 1

{
    "Reasoning": "As someone who has consistently voted in the past and believes in the importance of being an active participant in the democratic process, I plan to vote in the 2022 midterm election. Additionally, as a conservative individual, I believe that it is important to support candidates who align with my values and beliefs, and the midterm election provides an opportunity to do so. Therefore, my answer is Yes.",
    "Answer": "Yes"
}
{
    "Reasoning": "As someone who has consistently voted in the past and believes in the importance of being an active participant in the democratic process, I plan to vote in the 2022 midterm election. While I typically vote for the Republican Party and their candidates, I always take the time to research each candidate and their policies before casting my vote. Additionally, as a conservative living in rural Montana, I believe it is important to have my voice heard and to support candidates who share my values and beli

{
    "Reasoning": "As a conservative, I believe in upholding the Constitution and the principles it was founded upon. The Electoral College was put in place to ensure that all states, regardless of size or population, have a say in the election of the president. While I understand the argument for amending the Constitution to allow the candidate who receives the most votes to win, I believe that this would give too much power to heavily populated urban areas and would not accurately represent the views and values of the entire country. Therefore, I believe we should keep the current system where the candidate who wins the Electoral College vote wins.",
    "Answer": "Keep the current system"
}
{
    "Reasoning": "As a moderate with no strict political affiliation, I believe that the current system of the Electoral College has its pros and cons. While it can give smaller states a greater voice in the election, it can also result in a candidate winning the presidency without receiving t

{
    "Reasoning": "As a moderate individual with no strict party affiliation, I tend to evaluate politicians based on their job performance and how well they align with my values and beliefs. I have not closely followed Congressperson Matt Rosendale's job performance, but based on the limited information I have, I do not have any major issues with his performance. Therefore, I would say that I approve of his job performance at this point in time.",
    "Answer": "Approve"
}
{
    "Reasoning": "As a conservative with a history of voting for the Republican Party, I am likely to approve of Matt Rosendale's job performance as he is a member of the Republican Party. Additionally, as a resident of rural Montana, I may feel that Rosendale represents my interests and values in Congress. However, I do take the time to research each candidate and their policies before casting my vote, so I am not blindly approving of Rosendale's job performance without considering his actions and policies.",
  

{
    "Reasoning": "As a conservative individual who believes in limited government and personal responsibility, I generally distrust the ability of representatives to govern efficiently and in a non-divisive manner. I believe that government intervention often leads to inefficiencies and waste, and that politicians are often more concerned with their own interests than the interests of their constituents. Additionally, the current political climate has become increasingly polarized and divisive, which further erodes my trust in the ability of representatives to govern effectively. However, I do not completely distrust all representatives and acknowledge that there may be some who are genuinely committed to serving their constituents and working towards the greater good.",
    "Answer": "Distrust"
}
Starting Question: 15

{
    "Reasoning": "As a conservative living in rural Montana, I generally trust my local police and sheriff's department. I believe in law and order and support the 

{
    "Reasoning": "As a resident of urban Montana, I have seen firsthand the impact of mental health issues on individuals and communities. While I cannot speak for the entire state, I do believe that there is a mental health crisis in Montana. The state has a high suicide rate and limited access to mental health resources, particularly in rural areas. Additionally, the COVID-19 pandemic has exacerbated existing mental health issues and created new ones. Therefore, I agree that there is a mental health crisis in Montana.",
    "Answer": "Agree"
}
Starting Question: 19

{
    "Reasoning": "As a software developer living in urban Montana, I spend a lot of time indoors due to my occupation. While I do not suffer from depression myself, I understand that it is a prevalent issue in Montana. I believe that engaging in fun indoor activities can have a positive impact on mental health and could potentially help reduce the depression rate in Montana. Additionally, as someone who consumes a mod

{
    "Reasoning": "As someone who resides in urban Montana, I have a deep appreciation for the natural beauty of this state. I believe that public lands are an important resource that should be protected for future generations to enjoy. Additionally, as a moderate, I believe in finding a balance between economic development and environmental protection. Protecting public lands can have economic benefits, such as attracting tourism and outdoor recreation, while also preserving the natural beauty of the state. Therefore, I agree that Montana officials should work to protect public lands.",
    "Answer": "Agree"
}
Starting Question: 23

{
    "Reasoning": "As a conservative with a high school education level, I tend to hold pro-life beliefs. Additionally, I am a member of the Republican Party and have consistently voted for their candidates in the past. Therefore, I agree with the Supreme Court's decision to overturn Roe v. Wade.",
    
    "Answer": "Agree"
}
{
    "Reasoning": "As a co

{
    "Reasoning": "As a conservative individual, I believe in personal responsibility and accountability for one's actions. I also believe in the importance of law and order in society. In cases of convicted murder, I believe that the death penalty can serve as a just punishment for the most heinous crimes. However, I also recognize that the justice system is not perfect and there is always a risk of wrongful convictions. Therefore, I support the use of the death penalty in cases where there is irrefutable evidence of guilt and the crime is particularly heinous, but I also believe that there should be strict safeguards in place to ensure that innocent individuals are not wrongly executed.",
    "Answer": "Agree"
}
{
    "Reasoning": "As a conservative with a belief in personal responsibility, I believe that individuals should be held accountable for their actions. In cases of murder, I believe that the death penalty is an appropriate punishment for the taking of another person's life.

In [34]:
class SurveyResponseEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, SurveyResponse):
            return {
                "person": obj.person,
                "survey_responses": obj.survey_responses
            }
        return super().default(obj)

    # Save responses as JSON
with open("survey_responses.json", "w") as file:
    json.dump(SURVEY_RESPONSES, file, cls=SurveyResponseEncoder, indent=4)


# Analyze Results

In [44]:
resps_out = {}
for i in range(len(questions)):
    resps_out[i+1] = []

for i in range(len(questions)):
    for s in SURVEY_RESPONSES:
        
        try:
            resp_data = json.loads(s.survey_responses[str(i+1)])
            resp_val = resp_data['Answer']
            resps_out[i+1].append(resp_val)   
        except Exception as e:
            print("Invalid Format", str(e))
            continue

In [ ]:
def count_responses(responses):
    # Count occurrences of each unique value
    counts = {}
    total_responses = len(responses)

    for response in responses:
        if response not in counts:
            counts[response] = 0
        counts[response] += 1

    # Print counts and percentages
    print("Counts of each response:")
    for response, count in counts.items():
        percentage = (count / total_responses) * 100
        print(f"{response}: {count} ({percentage:.2f}%)")

In [48]:
for 

['Disapprove', 'Approve', 'Disapprove']

In [41]:
for s in SURVEY_RESPONSES:
    print(s.survey_responses)

{'1': '{\n    "Reasoning": "As someone who has consistently voted in the past and believes in the importance of being an active participant in the democratic process, I plan to vote in the 2022 midterm election. While I typically vote for the Republican Party and their candidates, I always take the time to research each candidate and their policies before casting my vote. Additionally, as a conservative living in rural Montana, I believe it is important to have my voice heard and to support candidates who share my values and beliefs.",\n    "Answer": "Yes"\n}', '2': '{\n    "Reasoning": "As a conservative with a history of voting for the Republican Party, I am likely to have a negative view of President Joe Biden\'s performance in office. Additionally, my media consumption habits are likely to expose me to conservative-leaning news outlets that may criticize the president\'s policies and decisions. However, as someone who takes the time to research each candidate and their policies bef

In [ ]:
def thread_ask_question(i, question):
    global total_prompt_tokens, total_completion_tokens
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(personas[i]))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question.format_for_gpt())
    print(persona_role, question_in)
    return ""
    try:
        resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.25, tokens = 300)
    except:
        return "",0
    
    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']
    
    print(i, resp)
    return resp


In [18]:
responses = []
question = "Do you plan to vote in the 2022 midterm election?"

question = "At this point in time, do you approve or disapprove of President Joe Biden's performance in office?\nApprove\nDisapprove\nNeither Approve/Disapprove"

question = "Do you approve or disapprove of President Joe Biden's student loan forgiveness plan?\nApprove\nDisapprove\nNeither Approve/Disapprove"


def thread_func(i):
    global total_prompt_tokens, total_completion_tokens
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(personas[i]))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question)
    #print(persona_role, question_in)
    try:
        resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.25, tokens = 300)
    except:
        return "",0
    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']
    print(i, resp)
    return resp

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    responses = list(executor.map(thread_func, range(30)))

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

2 {
  "Reasoning": "As a 41-year-old conservative software developer, I value personal responsibility and limited government intervention. I believe that individuals should be responsible for their own financial decisions, including taking out student loans. Additionally, I am concerned that forgiving student loans could lead to increased government spending and potentially higher taxes. My conservative political affiliation also influences my opinion on this matter, as I generally align with the Republican Party's stance on fiscal responsibility.",
  "Answer": "Disapprove"
}
1 {
  "Reasoning": "As a 31-year-old marketing manager with a Bachelor's Degree, I understand the burden of student loans and the impact it can have on individuals' lives. I believe in the importance of education and providing equal opportunities for everyone to succeed. While I am a moderate politically, I do appreciate the efforts to address the student loan crisis. However, I also understand that there are conc

11 {
  "Reasoning": "As a 29-year-old agricultural engineer with a Bachelor's Degree or higher, I have likely experienced the burden of student loans firsthand. However, my political affiliation as a right-leaning individual and my belief in personal responsibility and limited government interference may lead me to question the long-term effects of such a plan. I am concerned about the potential consequences of forgiving student loans on the economy, as well as the fairness of the policy for those who have already paid off their loans or did not attend college. Additionally, my media consumption, which includes conservative outlets like Fox News and The Daily Wire, may influence my opinion on this matter.",
  "Answer": "Disapprove"
}
16 {
  "Reasoning": "As a 57-year-old conservative from Montana, I believe in personal responsibility and limited government intervention. I have a high school education and work as a retail sales associate, so I did not take out student loans for higher e

18 {
  "Reasoning": "As a 48-year-old truck driver with a high school education, I have not directly benefited from higher education, and I am not burdened by student loan debt. My conservative political affiliation leads me to believe in limited government intervention and personal responsibility. I worry that forgiving student loans might encourage irresponsible borrowing in the future and place an unfair burden on taxpayers, including working-class individuals like myself. Additionally, I consume conservative media outlets like Fox News and The Blaze, which often criticize President Joe Biden's policies, further influencing my opinion on this matter.",
  "Answer": "Disapprove"
}
23 {
  "Reasoning": "As a 43-year-old white male with a conservative political affiliation, I generally believe in smaller government, lower taxes, and individual freedom and responsibility. I am a high school graduate and a construction worker, so I did not have to deal with student loans myself. However, I

# Process Responses

In [21]:
def count_answers(strings):
    count_disapprove = 0
    count_approve = 0
    count_neither = 0

    for string in strings:
        if string == "Disapprove":
            count_disapprove += 1
        elif string == "Approve":
            count_approve += 1
        elif string == "Neither Approve/Disapprove":
            count_neither += 1

    return count_disapprove, count_approve, count_neither


In [22]:
answers = []
for resp in responses:
    try:
        data = json.loads(resp)
    except:
        print("failed")
        continue
    answers.append(data['Answer'])
    
disapprove_count, approve_count, neither_count = count_answers(answers)
print("Disapprove count:", disapprove_count)
print("Approve count:", approve_count)
print("Neither count:", neither_count)

Disapprove count: 19
Approve count: 9
Neither count: 2


In [78]:
responses_2 = []
#question = "Do you plan to vote in the 2024 midterm election?\n Yes\n No\n Undecided\n Please note, you are allowed to say no and not voted"

question = "At this point in time, do you approve or disapprove of President Joe Biden's performance in office?\nApprove\nDisapprove\nNeither Approve/Disapprove"

#question = "Do you approve or disapprove of President Joe Biden's student loan forgiveness plan?\nApprove\nDisapprove\nNeither Approve/Disapprove"

total_prompt_tokens = 0
total_completion_tokens = 0


def thread_func(i):
    global total_prompt_tokens, total_completion_tokens
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(personas[i]))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question)
    #print(persona_role, question_in)
    resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.1, tokens = 300)
    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']
    print(i, resp)
    return resp

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    responses_2 = list(executor.map(thread_func, range(len(personas))))

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

9 {
    "Reasoning": "As Lisha Patterson, a conservative individual, I tend to disagree with President Joe Biden's policies and political views. Additionally, I consume media outlets that cater to conservative political views, which often criticize and oppose President Biden's actions. Therefore, I would disapprove of President Joe Biden's performance in office at this point in time.",
    "Answer": "Disapprove"
}
2 {
    "Reasoning": "As Sean Foster, a conservative individual who tends to vote for Republican candidates, I am likely to disapprove of President Joe Biden's performance in office. Additionally, my media consumption habits may expose me to news outlets that are critical of the current administration. However, I also believe in researching and understanding the policies and actions of elected officials before forming an opinion. Therefore, I will review President Biden's performance in office based on his policies and actions that align with my political beliefs and values."

12 {
    "Reasoning": "As Grace Davis, a left-leaning individual who prioritizes issues such as social justice, healthcare, and the environment, I approve of President Joe Biden's performance in office. He has taken steps to address these issues, such as signing executive orders to combat climate change and expand access to healthcare. Additionally, his administration has made efforts to promote diversity and inclusion, which aligns with my values. While there may be areas where I disagree with his policies, overall I believe that he is moving in the right direction and making progress on important issues.",
    "Answer": "Approve"
}
13 {
    "Reasoning": "As Felicity Ross, a conservative individual who prioritizes smaller government, lower taxes, and individual freedoms, I am likely to disapprove of President Joe Biden's performance in office. Additionally, I tend to vote for conservative candidates who align with my political beliefs and values, and President Biden's policies and act

27 {
    "Reasoning": "As Shannon Garcia, a right-leaning individual, I am likely to have some disagreements with President Joe Biden's policies and political views. However, as a responsible voter, I believe it is important to evaluate a politician's performance based on their actions and accomplishments rather than just their party affiliation. Based on my media consumption and personal research, I am aware that President Biden has taken several actions to address issues such as the COVID-19 pandemic, climate change, and social justice. While I may not agree with all of his policies, I believe that he is making an effort to address important issues and move the country forward. Therefore, I approve of President Joe Biden's performance in office at this point in time.",
    "Answer": "Approve"
}
26 {
    "Reasoning": "As Justin Martin, I am a moderate with no strong political affiliations. Therefore, I am likely to evaluate President Joe Biden's performance based on his policies and a

39 {
    "Reasoning": "As Kachina Brown, a conservative rancher from rural Montana, I tend to lean towards conservative policies and values. Therefore, I am more likely to disapprove of President Joe Biden's performance in office. Additionally, I tend to consume media outlets that align with my conservative political affiliation, which may also influence my opinion of the President's performance. However, I also believe in being an informed voter and researching each candidate and their policies before making a decision. Therefore, I will consider the actions and policies of President Biden before making a decision on whether to approve or disapprove of his performance in office.",
    "Answer": "Disapprove"
}
41 {
    "Reasoning": "As a left-leaning individual, I am more likely to approve of President Joe Biden's performance in office. Additionally, I am interested in policies that promote social justice, equality, and progressive values, which align with many of the policies that Pre

53 {
    "Reasoning": "As William Gonzales, a conservative individual who values smaller government and individual freedoms, I am likely to disapprove of President Joe Biden's performance in office. Based on my media consumption and influential figures, I am likely to consume media outlets that are critical of President Biden's policies and actions. Additionally, my voting history suggests that I tend to vote for conservative candidates who align with my political beliefs and values. Therefore, I am likely to disapprove of President Biden's performance in office due to policy disagreements and ideological differences.",
    
    "Answer": "Disapprove"
}
49 {
    "Reasoning": "As Sefu Bobcat, I tend to align myself with the Republican Party and conservative values. Therefore, I am more likely to disapprove of President Joe Biden's performance in office. Additionally, I consume media outlets that cater to conservative audiences, which may have a negative bias towards President Biden. How

63 {
    "Reasoning": "As Doris Howard, a liberal who values equal rights, access to affordable healthcare, and protecting the environment, I approve of President Joe Biden's performance in office. He has taken steps to address these issues, such as signing the American Rescue Plan Act to provide economic relief during the COVID-19 pandemic, expanding access to healthcare, and rejoining the Paris Climate Agreement. Additionally, as someone who values being an informed voter and making decisions based on facts, I appreciate President Biden's commitment to science and evidence-based policies. While there may be areas where I disagree with his policies, overall I believe he is working towards creating a more just and equitable society.",
    "Answer": "Approve"
}
66 {
    "Reasoning": "As Raymond Griffin, a left-leaning individual, I am likely to approve of President Joe Biden's performance in office. I prioritize issues such as healthcare and workers' rights, which align with Biden's pol

74 {
    "Reasoning": "As a conservative with a history of voting for the Republican Party, it is likely that Haley Coleman would disapprove of President Joe Biden's performance in office. Additionally, her media consumption habits suggest that she may consume conservative news sources that are critical of the current administration. However, it is important to note that Haley Coleman mentions that she does not blindly vote for any candidate just because they are Republican and that she always researches and evaluates their positions and track record before casting her vote. Therefore, it is possible that she may have some specific policy disagreements with the current administration but may also approve of some of their actions. Ultimately, it is difficult to say for certain without more information.",
    "Answer": "Neither Approve/Disapprove"
}
77 {
    "Reasoning": "As Violet Reed, I am a conservative and have a history of voting for the conservative party or candidate. Based on my

91 {
    "Reasoning": "As Isaac Smith, I am a conservative and have always supported the Republican Party. Based on my political affiliation, I am likely to disapprove of President Joe Biden's performance in office. Additionally, I have always believed in smaller government and lower taxes, which are not values that align with the current administration's policies. However, as a retired rancher, I am also concerned about issues affecting the agriculture industry, such as trade policies and environmental regulations. I would need to research more about the current administration's policies on these issues before making a final decision.",
    "Answer": "Disapprove"
}
90 {
    "Reasoning": "As Alexander Baker, a conservative individual who prioritizes smaller government, lower taxes, and a strong national defense, I am likely to disapprove of President Joe Biden's performance in office. President Biden's policies and actions, such as his proposed tax increases and his handling of the Afg

In [79]:
answers2 = []
for resp in responses_2:
    data = json.loads(resp)
    answers2.append(data['Answer'])
    
disapprove_count, approve_count, neither_count = count_answers(answers2)
print("Disapprove count:", disapprove_count)
print("Approve count:", approve_count)
print("Neither count:", neither_count)

Disapprove count: 52
Approve count: 36
Neither count: 12


In [80]:
responses_3 = []
#question = "Do you plan to vote in the 2024 midterm election?\n Yes\n No\n Undecided\n Please note, you are allowed to say no and not voted"

question = "Do you agree or disagree that there is a mental health crisis in Montana?\nAgree\nDisagree\nNeither Agree/Disagree"

#question = "Do you approve or disapprove of President Joe Biden's student loan forgiveness plan?\nApprove\nDisapprove\nNeither Approve/Disapprove"

total_prompt_tokens = 0
total_completion_tokens = 0


def thread_func(i):
    global total_prompt_tokens, total_completion_tokens
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(personas[i]))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question)
    #print(persona_role, question_in)
    resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-3.5-turbo' , role = persona_role, temp = 0.1, tokens = 300)
    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']
    print(i, resp)
    return resp

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    responses_3 = list(executor.map(thread_func, range(len(personas))))

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

3 {
    "Reasoning": "As a retired business owner living in urban Montana, I have seen the effects of mental health issues on individuals and the community. I am aware that Montana has one of the highest suicide rates in the country, and I believe that this is indicative of a larger mental health crisis. Additionally, I have seen firsthand how the lack of access to mental health resources can impact individuals and their families. Therefore, I agree that there is a mental health crisis in Montana.",
    "Answer": "Agree"
}
7 {
    "Reasoning": "As Charlotte Harris, I agree that there is a mental health crisis in Montana. While I do not have personal experience with mental health issues, I am aware that mental health resources and services are limited in rural areas like where I live. Additionally, the COVID-19 pandemic has had a significant impact on mental health, with many people experiencing increased stress, anxiety, and depression. I believe that addressing the mental health crisi

12 {
    "Reasoning": "As Grace Davis, I agree that there is a mental health crisis in Montana. While I do not have personal anecdotes related to this issue, I am aware of the high rates of suicide and substance abuse in Montana, particularly in rural areas. Additionally, I know that access to mental health services can be limited in Montana due to factors such as geographic location and lack of healthcare providers. As someone who believes in social justice and access to healthcare, I believe that addressing the mental health crisis in Montana is an important issue that requires attention and resources.",
    "Answer": "Agree"
}
17 {
    "Reasoning": "As Rosa Juarez, I am not sure if there is a mental health crisis in Montana. However, as a resident of a rural area, I know that access to mental health resources can be limited. Additionally, as someone with a lower income and education level, I know that mental health issues can be stigmatized and not always taken seriously. Therefore,

24 {
    "Reasoning": "As Koa Raccoon, I agree that there is a mental health crisis in Montana. Living in a rural area, I have seen firsthand the lack of access to mental health resources and support for individuals who are struggling with mental health issues. Additionally, as someone who has faced discrimination and marginalization, I understand the impact that systemic issues can have on mental health. Furthermore, my job as a construction worker can be physically and mentally demanding, and I have seen how the lack of support and resources can take a toll on individuals' mental health. Overall, based on my personal experiences and observations, I believe that there is a mental health crisis in Montana.",
    "Answer": "Agree"
}
25 {
    "Reasoning": "As Kristine Scott, I agree that there is a mental health crisis in Montana. In my role as a Veteran Outreach Coordinator, I have seen firsthand the impact that mental health issues can have on individuals and communities. Montana has a

43 {
    "Reasoning": "As Louis Kelly, I agree that there is a mental health crisis in Montana. While I may not have personal experience with mental health issues, I am aware of the high rates of suicide and substance abuse in Montana, particularly in rural areas like where I live. I also know that access to mental health services can be limited in rural areas, which can exacerbate the problem. Additionally, I believe that the COVID-19 pandemic has only made the mental health crisis worse, as people are dealing with increased stress, isolation, and uncertainty.",
    "Answer": "Agree"
}
42 {
    "Reasoning": "As Samuel Romero, I agree that there is a mental health crisis in Montana. While I may not have personal experience with mental health issues, I am aware of the impact that mental health can have on individuals and communities. As a construction worker, I have seen firsthand the toll that stress and anxiety can take on people in high-pressure jobs. Additionally, I am aware of the 

52 {
    "Reasoning": "As an 18-year-old retail sales associate from an urban area in Montana, I may not have a comprehensive understanding of the mental health situation in the state. However, based on my personal experiences and what I have heard from others, I believe that there is a mental health crisis in Montana. I have seen friends and family members struggle with mental health issues, and I know that access to mental health resources can be limited in rural areas. Additionally, I have heard about the high suicide rates in Montana, which is a clear indication that there are significant mental health challenges in the state. Therefore, I agree that there is a mental health crisis in Montana.",
    "Answer": "Agree"
}
54 {
    "Reasoning": "As Edith Brown, I agree that there is a mental health crisis in Montana. While I do not have any personal anecdotes related to mental health, I am aware of the challenges that individuals in my community face when it comes to accessing mental h

68 {
    "Reasoning": "As Jamie Ward, I agree that there is a mental health crisis in Montana. Although I do not have any specific personal anecdotes related to mental health, I am aware of the challenges that many individuals face in accessing mental health services, particularly in rural areas like Montana. Additionally, as a retired nurse, I understand the importance of mental health and the impact that it can have on overall well-being. Based on my media consumption and political affiliation, I am also likely to be aware of the current discussions and debates surrounding mental health policy and funding. Therefore, I believe that there is a mental health crisis in Montana and that more needs to be done to address this issue.",
    "Answer": "Agree"
}
71 {
    "Reasoning": "As Preston Taylor, I agree that there is a mental health crisis in Montana. While I may not have personal experience with mental health issues, I am aware that access to mental healthcare in rural areas can be li

86 {
    "Reasoning": "As Colin Lopez, I agree that there is a mental health crisis in Montana. While I may not have personal experience with mental health issues, I am aware that Montana has a high suicide rate and limited access to mental health resources. Additionally, the COVID-19 pandemic has likely exacerbated mental health issues for many people. I believe that it is important for our community and political leaders to prioritize mental health and work towards increasing access to resources and reducing the stigma surrounding mental health issues.",
    "Answer": "Agree"
}
88 {
    "Reasoning": "As a retired farmer living in rural Montana, I have seen firsthand the challenges that rural residents face in accessing quality healthcare. Mental health services are often limited in rural areas, and many people may not have access to the care they need. Additionally, the isolation and lack of social support that can come with living in a rural area may contribute to mental health issu

Error communicating with OpenAI: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 04f94693a3d78cc34840f4c42c046c66 in your message.)
85 {
    "Reasoning": "As Madison Perez, I agree that there is a mental health crisis in Montana. As a retired veteran, I have seen firsthand the impact that mental health issues can have on individuals and their families. Additionally, as a member of a local senior citizens group, I have heard stories from others in my community about the challenges they have faced in accessing mental health services. Based on my personal experiences and the information available, I believe that there is a significant need for increased resources and support for mental health in Montana.",
    "Answer": "Agree"
}
Total Prompt Tokens: 118151
Total Completion Tokens: 12195

Token Cost = $ 0.26069200000000003


In [2]:
def dict_to_paragraph(persona, residence):
    priorities = persona["Top Priorities"]
    party = random_party()
    try:
        pain_points = persona["Pain points"]
    except:
        pain_points = persona["Pain Points"]
    
    return (f"Your name is {persona['Name']}, you are a {persona['Age']} year old {persona['Ethnicity']} {persona['Gender']}.\n"
            f"You live in {residence} and your annual income is ${persona['Income']}.\n"
            f"Your marital status is {persona['Marital Status']}, and your highest level of education is {persona['Education Level']}.\n"
            f"You work as a {persona['Occupation']}. You are described as '{persona['Description']}'.\n"
            f"Your top priorities are {priorities}. You are facing some challenges, including {pain_points}.\n"
            f"Your political affiliation/party is {party}")

In [14]:
def count_answers_agree(strings):
    count_disapprove = 0
    count_approve = 0
    count_neither = 0

    for string in strings:
        if string == "Disagree":
            count_disapprove += 1
        elif string == "Agree":
            count_approve += 1
        elif string == "Neither Agree/Disagree":
            count_neither += 1

    return count_disapprove, count_approve, count_neither

In [83]:
answers3 = []
for resp in responses_3:
    data = json.loads(resp)
    answers3.append(data['Answer'])
    
disapprove_count, approve_count, neither_count = count_answers_agree(answers3)
print("Disapprove count:", disapprove_count)
print("Approve count:", approve_count)
print("Neither count:", neither_count)

Disapprove count: 0
Approve count: 100
Neither count: 0


In [94]:
def person_to_string(person):
    persona_string = []
    for key, value in person.items():
        # Some keys require specific phrasing to form a directive sentence
        if key == 'Name':
            persona_string.append(f"Your name is {value}.")
        elif key == 'Age':
            persona_string.append(f"You are {value} years old.")
        elif key == 'Ethnicity':
            persona_string.append(f"You identify as {value}.")
        elif key == 'Gender':
            persona_string.append(f"You are a {value} individual.")
        elif key == 'Income':
            persona_string.append(f"You make an annual income of around {value} USD.")
        elif key == 'Education Level':
            persona_string.append(f"Your highest level of education is {value}.")
        elif key == 'Political Affiliation':
            persona_string.append(f"You lean towards the {value} politically.")
        elif key == 'Geographic Location':
            persona_string.append(f"You currently live in {value}.")
        elif key == 'Veteran Status':
            persona_string.append(f"You are {value}.")
        elif key == 'Occupation':
            persona_string.append(f"You work as a {value}.")
        else:
            # The remaining keys don't require specific phrasing, so we can just add their values
            # These are written in first person so we can use them directly
            persona_string.append(value)
    
    # Join all elements of persona_string into a single string, separated by space
    return " ".join(persona_string)

In [24]:
responses_4 = []
#question = "Do you plan to vote in the 2024 midterm election?\n Yes\n No\n Undecided\n Please note, you are allowed to say no and not voted"

question = "Do you agree or disagree with the Supreme Court's decision to overturn Roe v. Wade?\nAgree\nDisagree\nNeither Agree/Disagree"

#question = "Do you approve or disapprove of President Joe Biden's student loan forgiveness plan?\nApprove\nDisapprove\nNeither Approve/Disapprove"

total_prompt_tokens = 0
total_completion_tokens = 0


def thread_func(i):
    global total_prompt_tokens, total_completion_tokens
    persona_role = GPT.open_file(f"Prompts/Polling/persona_role.txt").replace('<<PERSONA>>', str(personas[i]))
    question_in = GPT.open_file("Prompts/Polling/question_in.txt").replace('<<QUESTION>>', question)
    #print(persona_role, question_in)
    resp, usage = GPT.chat_gpt(question_in, engine = 'gpt-4' , role = persona_role, temp = 0.5, tokens = 600)
    total_completion_tokens += usage['completion_tokens']
    total_prompt_tokens += usage['prompt_tokens']
    print(i, resp)
    return resp

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    responses_4 = list(executor.map(thread_func, range(10)))

print("Total Prompt Tokens:", total_prompt_tokens)
print("Total Completion Tokens:", total_completion_tokens)
print()
print("Token Cost = $", str((total_prompt_tokens+total_completion_tokens)/1000*0.002))

5 {
  "Reasoning": "As Mae Foster, a 35-year-old conservative woman living in rural Montana, I have been raised with traditional values and a belief in individual freedoms. My conservative political affiliation and admiration for figures like Ronald Reagan have shaped my views on various issues, including abortion. I am concerned about the potential for government overreach and the erosion of individual freedoms.",
  "Answer": "Agree"
}
9 {
  "Reasoning": "As Lisha Patterson, I am a conservative and I believe in limited government and individual responsibility. I am also a Black woman who has experienced discrimination and inequality, so I understand the importance of equal rights and opportunities. When it comes to the Supreme Court's decision to overturn Roe v. Wade, I have to consider my conservative values and my personal experiences.",
  "Answer": "Agree"
}
8 {
  "Reasoning": "As a conservative, I tend to lean towards limited government intervention and upholding the Constitution.

In [26]:
answers4 = []
for resp in responses_4:
    try:
        data = json.loads(resp)
        answers4.append(data['Answer'])
    except:
        continue
    
disapprove_count, approve_count, neither_count = count_answers_agree(answers4)
print("Disapprove count:", disapprove_count)
print("Approve count:", approve_count)
print("Neither count:", neither_count)

Disapprove count: 0
Approve count: 9
Neither count: 1
